In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Notes

- paging is auto set to 1000, so only 1000 records were shown per request, to fix this I looked in the API documentation, and under paging saw that you have to set the limit. I set it to be over what the total dataset contained so I could always return everything from the set. this is the ?limit=50000 addition to the endpoint

In [ ]:
endpoint = 'https://data.nashville.gov/resource/fuaa-r5cm.json?$limit=50000'

# 1. Which month had the most 'fire' searches:

## August

In [ ]:
params = {'year': '2016',
          'query_text': 'fire'}

In [ ]:
response = requests.get(endpoint, params = params)

In [ ]:
response

In [ ]:
res = response.json()

In [ ]:
res

In [ ]:
len(res)

In [ ]:
res[0].keys()

In [ ]:
fire_search = pd.DataFrame(res)

In [ ]:
fire_search

In [ ]:
fire_search = fire_search.sort_values('query_count', ascending = False)

In [ ]:
fire_search.dtypes

In [ ]:
fire_search['query_count'] = pd.to_numeric(fire_search['query_count'])

In [ ]:
fire_search.nlargest(3, 'query_count')

# 2. How many times was a query run more than 100 times in one month? 1000 times

In [ ]:
# forgot to change limit here, can do with parameters, should have noticed the result having 1000 rows which is sketchy,
# this affects 3 too since i was lazy and didnt make a new response
endpoint2 = 'https://data.nashville.gov/resource/fuaa-r5cm.json?$where=query_count > 100'

response2 = requests.get(endpoint2)

In [ ]:
response2

In [ ]:
response2.text

In [ ]:
res2 = response2.json()

In [ ]:
res2

In [ ]:
over_100 = pd.DataFrame(res2)

In [ ]:
over_100

# 3. How many times 'codes' was searched for than 100 times in a month? 39

## does this really need to be another API request, cant you just get this from the above dataframe, over_100?

In [ ]:
over_100.query_text.value_counts()

# 4. Make an API request that returns the entire Top 500 Monthly Searches dataset. Make a chart that shows the number of times "maps" was searched in a month across the entire time frame.

In [ ]:
response4 = requests.get(endpoint)

In [ ]:
response4

In [ ]:
res4 = response4.json()

In [ ]:
#before adding the limit =50000, this length was 1000, which was sketchy, so i looked for any
# data return limits in the api docs and fixed it. now it looks right
len(res4)

In [ ]:
res4

In [ ]:
top_500 = pd.DataFrame(res4)

In [ ]:
top_500

In [ ]:
top_500.year.unique()

In [ ]:
# gonna want to use month_name and not month
top_500.isna().sum()

In [ ]:
maps = top_500.loc[top_500.query_text == 'maps'].reset_index(drop = True)

In [ ]:
maps.head()

In [ ]:
maps.dtypes

In [ ]:
maps['query_count'] = pd.to_numeric(maps['query_count'])

In [ ]:
maps.dtypes

In [ ]:
maps_by_month = maps.groupby('month_name')['query_count'].sum().reset_index()

In [ ]:
maps_by_month

In [ ]:
# make this nicer later, for now i just want to get some practice with API stuff
sns.barplot(data=maps_by_month, x='month_name', y='query_count')

# 5. Pull data from hubnash 311, how many rows can you return? Compared to top 500 data?

## The limit for both datasets were automatically set at 1000, but this can be changed. The top 500 data had ~ 50,000 rows, while the 311 data has ~ 1066000 rows

In [ ]:
# this limit isnt high enough, but it takes forever to set it to 1070000 so just leave it here
endpoint311 = 'https://data.nashville.gov/resource/7qhx-rexh.json?$limit=50000'

In [ ]:
response5 = requests.get(endpoint311)

In [ ]:
response5

In [ ]:
res5 = response5.json()

In [ ]:
res5

In [ ]:
len(res5)

# 6. Find 2 new data sets on data.nashville.gov, make API requests to pull the data, and do an analysis that combines the data sets.

In [ ]:
endpointevents = 'https://data.nashville.gov/resource/vygj-v677.json?$limit=5000'

In [ ]:
enpointwifi = 'https://data.nashville.gov/resource/4ugp-s85t.json'

In [ ]:
response_events = requests.get(endpointevents)

In [ ]:
res_events = response_events.json()

In [ ]:
res_events

In [ ]:
events = pd.DataFrame(res_events)

In [ ]:
events.head()

In [ ]:
response_wifi = requests.get(enpointwifi)

In [ ]:
res_wifi = response_wifi.json()

In [ ]:
res_wifi

In [ ]:
wifi = pd.DataFrame(res_wifi)

In [ ]:
wifi.head(2)

In [ ]:
wifi = wifi.drop(columns = [':@computed_region_c9xn_skx3', ':@computed_region_v3ji_vzam', ':@computed_region_gisn_y5cm', ':@computed_region_f73m_vb2k', ':@computed_region_sjpq_96s8', ':@computed_region_cfa7_hbpz'])

In [ ]:
wifi = wifi.rename(columns = {'site_name': 'location'})

In [ ]:
wifi.head()

In [ ]:
events.location.unique()

In [ ]:
wifi.location.unique()

In [ ]:
events_wifi = pd.merge(events, wifi, on = ['location'], how = 'left')

In [ ]:
events.shape

In [ ]:
wifi.shape

In [ ]:
events_wifi

In [ ]:
events_wifi.isna().sum()

In [ ]:
events_wifi_inner = pd.merge(events, wifi, on = ['location'], how = 'inner')

## so this did not have the overlap i thought it would, maybe i could get a count of attendance on the inner join and compare that to just the events df, and see if there is a higher attendance, or more events around locations with public wifi?

In [ ]:
events_wifi_inner

# 7. Compare between data on LA vs Sonoma employment statistics, what industries are most prevalent in big city vs wine country?

In [ ]:
endpointla = 'https://data.edd.ca.gov/resource/tzq9-9iaq.json?$limit=80000'

In [ ]:
responsela = requests.get(endpointla)

In [ ]:
resla = responsela.json()

In [ ]:
resla

In [ ]:
la_stats = pd.DataFrame(resla)

In [ ]:
la_stats.head(3)

In [ ]:
endpointsonoma = 'https://data.edd.ca.gov/resource/k7as-d5kv.json?$limit=20000'

In [ ]:
responseso = requests.get(endpointsonoma)

In [ ]:
resso = responseso.json()

In [ ]:
sonoma_stats = pd.DataFrame(resso)

In [ ]:
sonoma_stats.head()

In [ ]:
la_sonoma = pd.concat([la_stats, sonoma_stats])

In [ ]:
la_sonoma.head()

In [ ]:
la_stats.shape

In [ ]:
la_stats.dtypes

In [ ]:
sonoma_stats.shape

In [ ]:
la_stats['current_employment'] = pd.to_numeric(la_stats['current_employment'])

In [ ]:
sonoma_stats['current_employment'] = pd.to_numeric(sonoma_stats['current_employment'])

In [ ]:
la_stats.dtypes

In [ ]:
sonoma_stats.dtypes

In [ ]:
sonoma_stats = sonoma_stats.loc[sonoma_stats.seasonally_adjusted == 'N']

In [ ]:
la_stats = la_stats.loc[la_stats.seasonally_adjusted == 'N']

In [ ]:
la_sonoma = la_sonoma.loc[la_sonoma.seasonally_adjusted == 'N']

In [ ]:
la_sonoma.industry_title.unique()

In [ ]:
# to get a group by into a dataframe
la_by_year = la_stats.groupby(['year', 'industry_title'])['current_employment'].mean()

In [ ]:
type(la_by_year)

In [ ]:
la_by_year = pd.DataFrame(la_by_year).reset_index()

In [ ]:
la_2001 = la_by_year.loc[la_by_year.year == '2001']

In [ ]:
la_2001.head()

In [ ]:
la_2001 = la_2001.sort_values(by = 'current_employment', ascending = False)

In [ ]:
# i dropped the first two industries because they were umbrella titles, 
# that contained other industries 
la_2001 = la_2001.drop(la_2001.index[[0,1]])

In [ ]:
la_2001 = la_2001.drop(la_2001.index[[0,1]])

## Industries with most employment in LA in 2001

In [ ]:
la_2001.head()

In [ ]:
la_2001.shape

In [ ]:
la_2021 = la_by_year.loc[la_by_year.year == '2021']

In [ ]:
la_2021 = la_2021.sort_values(by = 'current_employment', ascending = False)

In [ ]:
la_2021 = la_2021.drop(la_2021.index[[0,1]])

In [ ]:
la_2021 = la_2021.drop(la_2021.index[[0,1]])

In [ ]:
la_2021.shape

## Industries with most employment in LA in 2021

In [ ]:
la_2021.head()

In [ ]:
sonoma_by_year = sonoma_stats.groupby(['year', 'industry_title'])['current_employment'].mean()

In [ ]:
sonoma_by_year = pd.DataFrame(sonoma_by_year).reset_index()

In [ ]:
sonoma_2001 = sonoma_by_year.loc[sonoma_by_year.year == '2001']

In [ ]:
sonoma_2001.head()

In [ ]:
sonoma_2001 = sonoma_2001.sort_values(by = 'current_employment', ascending = False)

In [ ]:
sonoma_2001.head()

In [ ]:
sonoma_2001 = sonoma_2001.drop(sonoma_2001.index[[0,1]])

In [ ]:
sonoma_2001 = sonoma_2001.drop(sonoma_2001.index[[0,1]])

In [ ]:
sonoma_2001.head()

In [ ]:
sonoma_2021 = sonoma_by_year.loc[sonoma_by_year.year == '2021']

In [ ]:
sonoma_2021 = sonoma_2021.sort_values(by = 'current_employment', ascending = False)

In [ ]:
sonoma_2021.head()

In [ ]:
sonoma_2021 = sonoma_2021.drop(sonoma_2021.index[[0,1]])

In [ ]:
sonoma_2021 = sonoma_2021.drop(sonoma_2021.index[[0,1]])

In [ ]:
sonoma_2021.head()

In [ ]:
la_2001_top = la_2001.nlargest(10, 'current_employment')

In [ ]:
la_2001_top.head()

In [ ]:
la_2021_top = la_2021.nlargest(10, 'current_employment')

In [ ]:
la_2021_top.head()

In [ ]:
sonoma_2001_top = sonoma_2001.nlargest(10, 'current_employment')

In [ ]:
sonoma_2001_top.head()

In [ ]:
sonoma_2021_top = sonoma_2021.nlargest(10, 'current_employment')

In [ ]:
sonoma_2021_top.head()

## Top 10 industries by year and county

In [ ]:
plt.figure(figsize = (8,6))
sns.barplot(data = la_2001_top,
            orient ='h',
            x = 'current_employment',
            y = 'industry_title',
            hue = 'industry_title',
            palette = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink', 'grey', 'greenyellow', 'aqua'],
            edgecolor = 'black',
            dodge = False).set_title('LA County 2001 Top Industries')

plt.legend([],[],frameon=False)
plt.xlabel('Number of Jobs')
plt.ylabel('Industry')


In [ ]:
plt.figure(figsize = (8,6))
sns.barplot(data = la_2021_top,
            orient ='h',
            x = 'current_employment',
            y = 'industry_title',
           hue = 'industry_title',
            palette = ['blue', 'grey', 'orange', 'aqua', 'purple', 'red', 'pink', 'green', 'cornflowerblue', 'greenyellow'],
            edgecolor = 'black',
            dodge = False).set_title('LA County 2021 Top Industries')

plt.legend([],[],frameon=False)
plt.xlabel('Number of Jobs')
plt.ylabel('Industry')

In [ ]:
plt.figure(figsize = (8,6))
sns.barplot(data = sonoma_2001_top,
            orient ='h',
            x = 'current_employment',
            y = 'industry_title',
              hue = 'industry_title',
            palette = ['blue', 'green', 'orange', 'brown', 'red', 'pink', 'yellow', 'grey', 'aqua', 'yellowgreen'],
            edgecolor = 'black',
            dodge = False).set_title('Sonoma County 2001 Top Industries')

plt.legend([],[],frameon=False)
plt.xlabel('Number of Jobs')
plt.ylabel('Industry')

In [ ]:
plt.figure(figsize = (8,6))
sns.barplot(data = sonoma_2021_top,
            orient ='h',
            x = 'current_employment',
            y = 'industry_title',
               hue = 'industry_title',
            palette = ['blue', 'green', 'orange', 'grey', 'aqua', 'red', 'pink', 'purple', 'yellow', 'brown'],
            edgecolor = 'black',
            dodge = False).set_title('Sonoma County 2021 Top Industries')

plt.legend([],[],frameon=False)
plt.xlabel('Number of Jobs')
plt.ylabel('Industry')

### looking at the graphs above, i notced that there are not many differences between counties in top industries, except there are more leisure and hospitality jobs in la, and more retail trade jobs in sonoma

### It was more interesting to notice which industries grew between 2001 and 2021

healthcare and social assiatance, and educational and health services were both industries that had more jobs in the last 20 years.

what other industries had an overall increase in jobs. This will be affected by population growth, so I will just be looking at industries with the highest % of growth comapred to growth of other industries

In [ ]:
growth20_la = pd.merge(la_2001, la_2021, on = 'industry_title', how = 'inner')

In [ ]:
growth20_la.head()

In [ ]:
growth20_la = growth20_la.rename(columns = {'current_employment_x': 'employment_2001', 'current_employment_y': 'employment_2021'})

In [ ]:
growth20_la = growth20_la.drop(columns = ['year_x', 'year_y'])

In [ ]:
growth20_la.head()

In [ ]:
growth20_la['growth'] = growth20_la.employment_2021 / growth20_la.employment_2001 

In [ ]:
growth20_la.head()

In [ ]:
growth20_la = growth20_la.sort_values(by = 'growth', ascending = False)

## below are the industries in LA by growth in the last 20 years (make graph with top and bottom 5)

In [ ]:
top_growth_la = growth20_la.head()

In [ ]:
top_growth_la

In [ ]:
sns.barplot(data=top_growth_la, x='growth', y='industry_title', orient='h',
            hue='industry_title', 
            palette=['red', 'silver', 'silver', 'red', 'silver'],
            edgecolor='black',
            dodge = False).set_title('Top 5 Growth in Industry LA')

plt.xlabel('Growth')
plt.ylabel('Industry')
plt.legend([],[],frameon=False)

In [ ]:
bottom_growth_la = growth20_la.tail()

In [ ]:
bottom_growth_la

In [ ]:
sns.barplot(data=bottom_growth_la, x='growth', y='industry_title', orient='h',
            hue='industry_title', 
            palette=['silver', 'silver', 'silver', 'silver', 'red'],
            edgecolor='black',
            dodge = False).set_title('Bottom 5 Growth in Industry LA')

plt.xlabel('Growth')
plt.ylabel('Industry')
plt.legend([],[],frameon=False)

In [ ]:
growth20_sonoma = pd.merge(sonoma_2001, sonoma_2021, on = 'industry_title', how = 'inner')

In [ ]:
growth20_sonoma = growth20_sonoma.rename(columns = {'current_employment_x': 'employment_2001', 'current_employment_y': 'employment_2021'})

In [ ]:
growth20_sonoma = growth20_sonoma.drop(columns = ['year_x', 'year_y'])

In [ ]:
growth20_sonoma.head()

In [ ]:
growth20_sonoma['growth'] = growth20_sonoma.employment_2021 / growth20_sonoma.employment_2001 

In [ ]:
growth20_sonoma.head()

In [ ]:
growth20_sonoma = growth20_sonoma.sort_values(by = 'growth', ascending = False)

## below are the industries in Sonoma by growth in the last 20 years (make graph with top and bottom 5)

In [ ]:
top_growth_sonoma = growth20_sonoma.head()

In [ ]:
top_growth_sonoma

In [ ]:
sns.barplot(data=top_growth_sonoma, x='growth', y='industry_title', orient='h',
            hue='industry_title', 
            palette=['silver', 'red', 'red', 'silver', 'silver'],
            edgecolor='black',
            dodge = False).set_title('Top 5 Growth in Industry Sonoma')

plt.xlabel('Growth')
plt.ylabel('Industry')
plt.legend([],[],frameon=False)

In [ ]:
bottom_growth_sonoma = growth20_sonoma.tail()

In [ ]:
bottom_growth_sonoma

In [ ]:
sns.barplot(data=bottom_growth_sonoma, x='growth', y='industry_title', orient='h',
            hue='industry_title', 
            palette=['silver', 'silver', 'red', 'silver', 'red'],
            edgecolor='black',
            dodge = False).set_title('Bottom 5 Growth in Industry Sonoma')

plt.xlabel('Growth')
plt.ylabel('Industry')
plt.legend([],[],frameon=False)

# employment flux by season in sonoma and la, for 5 prevalent industries.

## compare 2021 with 2020 (did covid stuff make notable changes here?)

not really

In [ ]:
la_stats5_21 = la_stats.loc[la_stats.industry_title.isin(['Educational and Health Services', 'Trade, Transportation and Utilities', 'Government', 'Goods Producing', 'Health Care and Social Assistance'])]

In [ ]:
la_stats5_21 = la_stats5_21.loc[la_stats5_21.year == '2021']

In [ ]:
la_stats5_21.head()

In [ ]:
sonoma_stats5_21 = sonoma_stats.loc[sonoma_stats.industry_title.isin(['Educational and Health Services', 'Trade, Transportation and Utilities', 'Government', 'Goods Producing', 'Health Care and Social Assistance'])]

In [ ]:
sonoma_stats5_21 = sonoma_stats5_21.loc[sonoma_stats5_21.year == '2021']

In [ ]:
sonoma_stats5_21.head()

In [ ]:
la5_2021 = la_stats5_21.groupby(['month', 'industry_title'])['current_employment'].mean()

In [ ]:
la5_2021 = pd.DataFrame(la5_2021).reset_index()

In [ ]:
la5_2021.head()

## change in employment in LA by month

## (I picked 5 prevalent industries to compare)

In [ ]:
#la 2021 flux by month
la_2021_grid = sns.FacetGrid(la5_2021, col = 'industry_title',
                             col_wrap = 2,
                             height = 5)

la_2021_grid.map(sns.barplot, 'current_employment', 'month', orient ='h')

la_2021_grid.set_xlabels('Number of Jobs')
la_2021_grid.fig.suptitle('Change in Employment by Month: LA 2021', y=1.03, fontsize = '20')

In [ ]:
la_stats5_20 = la_stats.loc[la_stats.industry_title.isin(['Educational and Health Services', 'Trade, Transportation and Utilities', 'Government', 'Goods Producing', 'Health Care and Social Assistance'])]

In [ ]:
la_stats5_20 = la_stats5_20.loc[la_stats5_20.year == '2020']

In [ ]:
la_stats5_20.head()

In [ ]:
la5_2020 = la_stats5_20.groupby(['month', 'industry_title'])['current_employment'].mean()

In [ ]:
la5_2020 = pd.DataFrame(la5_2020).reset_index()

In [ ]:
la5_2020.head()

In [ ]:
# la 2020 flux by month
la_2020_grid = sns.FacetGrid(la5_2020, col = 'industry_title',
                             col_wrap = 2,
                             height = 5)

la_2020_grid.map(sns.barplot, 'current_employment', 'month', orient ='h')

la_2020_grid.set_xlabels('Number of Jobs')
la_2020_grid.fig.suptitle('Change in Employment by Month: LA 2020', y=1.03, fontsize = '20')

## change in employment in Sonoma by month

## (I picked 5 prevalent industries to compare)

In [ ]:
sonoma_stats5_21 = sonoma_stats.loc[sonoma_stats.industry_title.isin(['Educational and Health Services', 'Trade, Transportation and Utilities', 'Government', 'Goods Producing', 'Health Care and Social Assistance'])]

In [ ]:
sonoma_stats5_21 = sonoma_stats5_21.loc[sonoma_stats5_21.year == '2021']

In [ ]:
sonoma_stats5_21.head()

In [ ]:
sonoma5_2021 = sonoma_stats5_21.groupby(['month', 'industry_title'])['current_employment'].mean()

In [ ]:
sonoma5_2021 = pd.DataFrame(sonoma5_2021).reset_index()

In [ ]:
sonoma5_2021.head()

In [ ]:
sonoma_2021_grid = sns.FacetGrid(sonoma5_2021, col = 'industry_title',
                             col_wrap = 2,
                             height = 5)

sonoma_2021_grid.map(sns.barplot, 'current_employment', 'month', orient ='h')

sonoma_2021_grid.set_xlabels('Number of Jobs')
sonoma_2021_grid.fig.suptitle('Change in Employment by Month: Sonoma 2021', y=1.03, fontsize = '20')

In [ ]:
sonoma_stats5_20 = sonoma_stats.loc[sonoma_stats.industry_title.isin(['Educational and Health Services', 'Trade, Transportation and Utilities', 'Government', 'Goods Producing', 'Health Care and Social Assistance'])]

In [ ]:
sonoma_stats5_20 = sonoma_stats5_20.loc[sonoma_stats5_20.year == '2020']

In [ ]:
sonoma5_2020 = sonoma_stats5_20.groupby(['month', 'industry_title'])['current_employment'].mean()

In [ ]:
sonoma5_2020 = pd.DataFrame(sonoma5_2020).reset_index()

In [ ]:
sonoma_2020_grid = sns.FacetGrid(sonoma5_2020, col = 'industry_title',
                             col_wrap = 2,
                             height = 5)

sonoma_2020_grid.map(sns.barplot, 'current_employment', 'month', orient ='h')

sonoma_2020_grid.set_xlabels('Number of Jobs')
sonoma_2020_grid.fig.suptitle('Change in Employment by Month: Sonoma 2020', y=1.03, fontsize = '20')